# Not Using Pauli Frame Update

In [627]:
from qiskit import __version__
print(__version__)

1.4.2


In [739]:
from qiskit import QuantumCircuit, ClassicalRegister
from qiskit.quantum_info import Statevector, state_fidelity, partial_trace, DensityMatrix, Pauli
from qiskit.quantum_info.operators import Operator
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit import transpile 
import numpy as np
from qiskit_aer.noise import NoiseModel, depolarizing_error, ReadoutError
from qiskit.circuit.controlflow import IfElseOp
from qiskit.circuit.library import XGate, ZGate
import matplotlib.pyplot as plt

In [726]:
import importlib
import steane_ec_decoder2
importlib.reload(steane_ec_decoder2)
from steane_ec_decoder2 import allstab_lookup

# Function for Encoding

In [676]:
def encoding(qc: QuantumCircuit):
    theta = np.arctan(np.sqrt((np.sqrt(5) - 1) / 2))
    amp_0 = np.cos(theta/2)
    amp_1 = np.sin(theta/2)
    qc.initialize([amp_0, amp_1], 0)
    for i in range(7):
        qc.id(i)
    for i in range(4, 7):
        qc.h(i)
    qc.cx(0, 1)
    qc.cx(0, 2)
    qc.cx(6, 0)
    qc.cx(6, 1)
    qc.cx(6, 3)
    qc.cx(5, 0)
    qc.cx(5, 2)
    qc.cx(5, 3)
    qc.cx(4, 1)
    qc.cx(4, 2)
    qc.cx(4, 3)

# Functions for Stabilizer Extraction 

In [692]:
def stabilizer6(qc: QuantumCircuit, first_qubit: int, cs: ClassicalRegister):
    # Measuring XXXXIII to qubit 7
    for i in range(4):
        qc.h(first_qubit+i)
        qc.cx(first_qubit+i, first_qubit+7)
        qc.h(first_qubit+i)
        
    # Measuring IXXIXXI to qubit 8
    for i in [1, 2, 4, 5]:
        qc.h(first_qubit+i)
        qc.cx(first_qubit+i, first_qubit+8)
        qc.h(first_qubit+i)
    
    # Measuring IIXXIXX to qubit 9
    for i in [2, 3, 5, 6]:
        qc.h(first_qubit+i)
        qc.cx(first_qubit+i, first_qubit+9)
        qc.h(first_qubit+i)
        
    # Measuring ZZZZIII to qubit 10
    for i in range(4):
        qc.cx(first_qubit+i, first_qubit+7)

    # Measuring IZZIZZI to qubit 11
    for i in [1, 2, 4, 5]:
        qc.cx(first_qubit+i, first_qubit+8)

    # Measuring IIZZIZZ to qubit 12
    for i in [2, 3, 5, 6]:
        qc.cx(first_qubit+i, first_qubit+9)
        
    # Measure
    qc.measure([first_qubit+i for i in range(7, 13)], cs)

In [693]:
def flag1(qc: QuantumCircuit, first_qubit: int, c1: ClassicalRegister):
    qc.h(first_qubit+8)
    
    for i in range(4):
        qc.h(first_qubit+i)
        
    for i in [0, 8, 1, 2, 8, 3]:
        qc.cx(first_qubit+i, first_qubit+7)
    
    for i in range(4):
        qc.h(first_qubit+i)
    
    qc.h(first_qubit+8)
    qc.measure([first_qubit+7, first_qubit+8], c1)

In [694]:
def flag2(qc: QuantumCircuit, first_qubit: int, c2: ClassicalRegister):
    qc.h(first_qubit+8)
    
    for i in [1, 2, 4, 5]:
        qc.h(first_qubit+i)
        
    for i in [1, 8, 2, 4, 8, 5]:
        qc.cx(first_qubit+i, first_qubit+7)
    
    for i in [1, 2, 4, 5]:
        qc.h(first_qubit+i)
        
    qc.h(first_qubit+8)
    qc.measure([first_qubit+7, first_qubit+8], c2)

In [695]:
def flag3(qc: QuantumCircuit, first_qubit: int, c3: ClassicalRegister):
    qc.h(first_qubit+8)
    
    for i in [2, 3, 5, 6]:
        qc.h(first_qubit+i)
    
    for i in [2, 8, 3, 5, 8, 6]:
        qc.cx(first_qubit+i, first_qubit+7)
        
    for i in [2, 3, 5, 6]:
        qc.h(first_qubit+i)
        
    qc.h(first_qubit+8)
    qc.measure([first_qubit+7, first_qubit+8], c3)

In [696]:
def flag4(qc: QuantumCircuit, first_qubit: int, c4: ClassicalRegister):
    qc.h(first_qubit+8)
    
    for i in [0, 8, 1, 2, 8, 3]:
        qc.cx(first_qubit+i, first_qubit+7)
        
    qc.h(first_qubit+8)
    qc.measure([first_qubit+7, first_qubit+8], c4)

In [697]:
def flag5(qc: QuantumCircuit, first_qubit: int, c5):
    qc.h(first_qubit+8)
        
    for i in [1, 8, 2, 4, 8, 5]:
        qc.cx(first_qubit+i, first_qubit+7)
        
    qc.h(first_qubit+8)
    qc.measure([first_qubit+7, first_qubit+8], c5)

In [699]:
def flag6(qc: QuantumCircuit, first_qubit: int, c6):
    qc.h(first_qubit+8)
    
    for i in [2, 8, 3, 5, 8, 6]:
        qc.cx(first_qubit+i, first_qubit+7)
        
    qc.h(first_qubit+8)
    qc.measure([first_qubit+7, first_qubit+8], c6)

# Function for QEC

In [708]:
def QEC(qc: QuantumCircuit, c1: ClassicalRegister, c2: ClassicalRegister, c3: ClassicalRegister, c4: ClassicalRegister, c5: ClassicalRegister, c6: ClassicalRegister, cs1: ClassicalRegister, cs2: ClassicalRegister, cs3: ClassicalRegister, cs4: ClassicalRegister, cs5: ClassicalRegister, cs6: ClassicalRegister):
    # Measure flag1 circuit
    flag1(qc, 0, c1)
    qc.reset([7,8])
    
    # Path 1, flag and syndrome are 0 for flag1
    do_nothing = QuantumCircuit(13)
    do_nothing.add_register(c1, c2, c3, c4, c5, c6, cs1, cs2, cs3, cs4, cs5, cs6)
    
    # Path 2, flag or syndrome are 1 for flag1
    all_stab1 = QuantumCircuit(13)
    all_stab1.add_register(c1, c2, c3, c4, c5, c6, cs1, cs2, cs3, cs4, cs5, cs6)
    stabilizer6(all_stab1, 0, cs1)
    
    first_if = IfElseOp((c1, 0), true_body=do_nothing, false_body=all_stab1)
    qc.append(first_if, list(range(13)), c1[:] + c2[:] + c3[:] + c4[:] + c5[:] + c6[:] + cs1[:] + cs2[:] + cs3[:] +cs4[:] + cs5[:] + cs6[:])
    
    # Measure flag2 circuit
    flag2(qc, 0, c2)
    qc.reset([7,8])
    
    # Path 1, flag and syndrome are 0 for flag2
    # Path 2, flag or syndrome are 1 for flag2
    all_stab2 = QuantumCircuit(13)
    all_stab2.add_register(c1, c2, c3, c4, c5, c6, cs1, cs2, cs3, cs4, cs5, cs6)
    stabilizer6(all_stab2, 0, cs2)
    
    second_if = IfElseOp((c2, 0), true_body=do_nothing, false_body=all_stab2)
    qc.append(second_if, list(range(13)), c1[:] + c2[:] + c3[:] + c4[:] + c5[:] + c6[:] + cs1[:] + cs2[:] + cs3[:] +cs4[:] + cs5[:] + cs6[:])
    
    # Measure flag3 circuit
    flag3(qc, 0, c3)
    qc.reset([7,8])
    
    # Path 1, flag and syndrome are 0 for flag3
    # Path 2, flag or syndrome are 1 for flag3
    all_stab3 = QuantumCircuit(13)
    all_stab3.add_register(c1, c2, c3, c4, c5, c6, cs1, cs2, cs3, cs4, cs5, cs6)
    stabilizer6(all_stab3, 0, cs3)
    
    third_if = IfElseOp((c3, 0), true_body=do_nothing, false_body=all_stab3)
    qc.append(third_if, list(range(13)), c1[:] + c2[:] + c3[:] + c4[:] + c5[:] + c6[:] + cs1[:] + cs2[:] + cs3[:] +cs4[:] + cs5[:] + cs6[:])
    
    # Measure flag4 circuit
    flag4(qc, 0, c4)
    qc.reset([7,8])
    
    # Path 1, flag and syndrome are 0 for flag4
    # Path 2, flag or syndrome are 1 for flag4
    all_stab4 = QuantumCircuit(13)
    all_stab4.add_register(c1, c2, c3, c4, c5, c6, cs1, cs2, cs3, cs4, cs5, cs6)
    stabilizer6(all_stab4, 0, cs4)
    
    fourth_if = IfElseOp((c4, 0), true_body=do_nothing, false_body=all_stab4)
    qc.append(fourth_if, list(range(13)), c1[:] + c2[:] + c3[:] + c4[:] + c5[:] + c6[:] + cs1[:] + cs2[:] + cs3[:] +cs4[:] + cs5[:] + cs6[:])
    
    # Measure flag5 circuit
    flag5(qc, 0, c5)
    qc.reset([7,8])
    
    # Path 1, flag and syndrome are 0 for flag5
    # Path 2, flag or syndrome are 1 for flag5
    all_stab5 = QuantumCircuit(13)
    all_stab5.add_register(c1, c2, c3, c4, c5, c6, cs1, cs2, cs3, cs4, cs5, cs6)
    stabilizer6(all_stab5, 0, cs5)
    
    fifth_if = IfElseOp((c5, 0), true_body=do_nothing, false_body=all_stab5)
    qc.append(fifth_if, list(range(13)), c1[:] + c2[:] + c3[:] + c4[:] + c5[:] + c6[:] + cs1[:] + cs2[:] + cs3[:] +cs4[:] + cs5[:] + cs6[:])
    
    # Measure flag6 circuit
    flag6(qc, 0, c6)
    qc.reset([7,8])
    
    # Path 1, flag and syndrome are 0 for flag6
    # Path 2, flag or syndrome are 1 for flag6
    all_stab6 = QuantumCircuit(13)
    all_stab6.add_register(c1, c2, c3, c4, c5, c6, cs1, cs2, cs3, cs4, cs5, cs6)
    stabilizer6(all_stab6, 0, cs6)
    
    sixth_if = IfElseOp((c6, 0), true_body=do_nothing, false_body=all_stab6)
    qc.append(sixth_if, list(range(13)), c1[:] + c2[:] + c3[:] + c4[:] + c5[:] + c6[:] + cs1[:] + cs2[:] + cs3[:] +cs4[:] + cs5[:] + cs6[:])
    

# Encoding + 1 Round of EC

In [709]:
noise_model = NoiseModel()

noise_model.add_all_qubit_quantum_error(depolarizing_error(0.005,1), ['id'])
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.005,1), ['h'])
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.05,2), ['cx'])

# 5% chance of flipping 0 <-> 1
readout_err = ReadoutError([[0.995, 0.005],  # P(measured 0 | actual 0), P(1 | 0)
                            [0.005, 0.995]]) # P(0 | 1), P(1 | 1)

# Apply to all qubits being measured
noise_model.add_readout_error(readout_err, [7]) 
noise_model.add_readout_error(readout_err, [8])
noise_model.add_readout_error(readout_err, [9])
noise_model.add_readout_error(readout_err, [10])
noise_model.add_readout_error(readout_err, [11])
noise_model.add_readout_error(readout_err, [12])

In [744]:
qc = QuantumCircuit(13)

# Classical Registers for flag1 measurements
c1 = ClassicalRegister(2, "c1")
cs1 = ClassicalRegister(6, "cs1")
# Classical Registers for flag2 measurements
c2 = ClassicalRegister(2, "c2")
cs2 = ClassicalRegister(6, "cs2")
# Classical Registers for flag3 measurements
c3 = ClassicalRegister(2, "c3")
cs3 = ClassicalRegister(6, "cs3")
# Classical Registers for flag4 measurements
c4 = ClassicalRegister(2, "c4")
cs4 = ClassicalRegister(6, "cs4")
# Classical Registers for flag5 measurements
c5 = ClassicalRegister(2, "c5")
cs5 = ClassicalRegister(6, "cs5")
# Classical Registers for flag6 measurements
c6 = ClassicalRegister(2, "c6")
cs6 = ClassicalRegister(6, "cs6")

qc.add_register(c1, cs1, c2, cs2, c3, cs3, c4, cs4, c5, cs5, c6, cs6)

encoding(qc)

QEC(qc, c1, c2, c3, c4, c5, c6, cs1, cs2, cs3, cs4, cs5, cs6)

qc.save_statevector(label='statevector_post', pershot=True, conditional=True)

backend = AerSimulator(noise_model=noise_model)
transpiled = transpile(qc, backend, optimization_level=0)
job = backend.run(transpiled, shots=1, memory=True)
result = job.result()
memory = result.get_memory()

print(result)
print(memory)

Result(backend_name='aer_simulator', backend_version='0.14.2', qobj_id='', job_id='75394de0-c576-4bd6-9978-1dd49e82a012', success=True, results=[ExperimentResult(shots=1, success=True, meas_level=2, data=ExperimentResultData(counts={'0x30b1609': 1}, memory=['0x30b1609'], statevector_post={'0x30b1609': [Statevector([2.74404926e-15+3.34117436e-01j,
             4.55099764e-31+2.94392336e-17j,
             0.00000000e+00+0.00000000e+00j, ...,
             0.00000000e+00+0.00000000e+00j,
             0.00000000e+00+0.00000000e+00j,
             0.00000000e+00+0.00000000e+00j],
            dims=(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2))]}), header=QobjExperimentHeader(creg_sizes=[['c1', 2], ['cs1', 6], ['c2', 2], ['cs2', 6], ['c3', 2], ['cs3', 6], ['c4', 2], ['cs4', 6], ['c5', 2], ['cs5', 6], ['c6', 2], ['cs6', 6]], global_phase=4.71238898038469, memory_slots=48, n_qubits=13, name='circuit-2853', qreg_sizes=[['q', 13]], metadata={}), status=DONE, seed_simulator=1920908839, metadata={'num_bind

In [745]:
corrections = allstab_lookup(memory)
print(corrections)

['IIIIZII', 'IIIIIII', 'IIIIIII', 'IIIIIII', 'IIIIIII', 'IIIIIII']


In [748]:
sv = result.data()['statevector_post']['0x30b1609'][0]
print(sv)

Statevector([2.74404926e-15+3.34117436e-01j,
             4.55099764e-31+2.94392336e-17j,
             0.00000000e+00+0.00000000e+00j, ...,
             0.00000000e+00+0.00000000e+00j,
             0.00000000e+00+0.00000000e+00j,
             0.00000000e+00+0.00000000e+00j],
            dims=(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2))


In [749]:
for correction in corrections:
    pauli13_label = correction + 'I'*6
    pauli13 = Pauli(pauli13_label)
    U = Operator(pauli13)
    sv = sv.evolve(U)

In [750]:
red_sv = partial_trace(sv, [7, 8, 9, 10, 11, 12])

In [751]:
display(red_sv.draw("latex"))

<IPython.core.display.Latex object>

In [752]:
test = QuantumCircuit(7)
encoding(test)
test = DensityMatrix(test)
display(test.draw("latex"))

<IPython.core.display.Latex object>

In [753]:
print(red_sv == test)

False
